In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("drug_check.csv")

In [3]:
df

,id,name,combination,nodenorm_id,has_rxcui,in_robokop
0,CHEBI:71255,ASENAPINE; ASENAPINE,FALSE,CHEBI:71255,False,True
1,DRUGBANK:DB14113,ANTIVENIN (LATRODECTUS MACTANS),FALSE,DRUGBANK:DB14113,True,False
2,MESH:D016899,INTERFERON-BETA,FALSE,MESH:D016899,True,False
3,RXCUI:1157355,ETHINYL ESTRADIOL; NORETHINDRONE,TRUE,RXCUI:1157355,True,False
4,RXCUI:1157364,ETHINYL ESTRADIOL; NORGESTIMATE,TRUE,RXCUI:1157364,True,False
...,...,...,...,...,...,...
785,RXCUI:1597612,CEFTOLOZANE; TAZOBACTAM,TRUE,RXCUI:1597612,True,False
786,RXCUI:2055981,ETHINYL ESTRADIOL; SEGESTERONE,TRUE,RXCUI:2055981,True,False
787,RXCUI:220751,ACETAMINOPHEN; ASPIRIN; CAFFEINE,TRUE,RXCUI:209468,True,False
788,RXCUI:369300,BENDROFLUMETHIAZIDE; NADOLOL,TRUE,NaN,NaN,False


In [7]:
#How many combination therapies? and how many in robokop?
df.groupby(["combination","in_robokop","has_rxcui"]).count()

id  name  nodenorm_id
combination in_robokop has_rxcui                        
FALSE       False      False      220   220          220
                       True       101   101          101
            True       False       16    16           16
                       True        44    44           44
False; True False      True        10    10           10
TRUE        False      False        1     1            1
                       True       361   361          361
            True       True        23    23           23
True; False False      False        1     1            1
                       True         7     7            7

So there are about 361 chemicals that are combination therapies not in RK and that also don't have an rxcui.

There are about 83 (of the total) that are in ROBOKOP.

There are 220 that are non-combination and not in RK and they dont have an RXCUI,
which leads me to think that there's a normalization issue with these and they might be cryptically in RK.

And about 100 that do have an RXCUI but still aren't in RK for some reason.

Here are the ones that don't have RXCUIs:

In [9]:
df[ (df["has_rxcui"] == False) & (df["in_robokop"] == False) & (df["combination"] == "FALSE") ]

,id,name,combination,nodenorm_id,has_rxcui,in_robokop
8,UNII:C7S8VWP5DH,PAMIDRONATE DISODIUM,FALSE,UNII:C7S8VWP5DH,False,False
11,PUBCHEM.COMPOUND:9824461,REMIMAZOLAM; REMIMAZOLAM,FALSE,PUBCHEM.COMPOUND:9824461,False,False
18,MESH:D022242,PNEUMOCOCCAL VACCINE,FALSE,MESH:D022242,False,False
24,UNII:6SN82Y9U73,FORODESINE HYDROCHLORIDE,FALSE,UNII:6SN82Y9U73,False,False
26,PUBCHEM.COMPOUND:71457944,COLISTIN SULFATE,FALSE,PUBCHEM.COMPOUND:71457944,False,False
...,...,...,...,...,...,...
768,PUBCHEM.COMPOUND:11305985,TOCOPHEROL ACETATE,FALSE,PUBCHEM.COMPOUND:11305985,False,False
771,UNII:4I27BB349X,ALKAVERVIR,FALSE,UNII:4I27BB349X,False,False
773,PUBCHEM.COMPOUND:157010649,ELTROMBOPAG CHOLINE,FALSE,PUBCHEM.COMPOUND:157010649,False,False
779,UNII:I69T4JE922,OLANEXIDINE GLUCONATE,FALSE,UNII:I69T4JE922,False,False


I think that a lot of these are normalization problems.  Because they don't have RXCUIs they aren't taking
advantage of drug/chemical conflation.  And in these cases we have some very related things in robokop. Examples;

Pamidronate Disodium: Pamidronic Acid (CHEBI:7903)

Remimazolam: Rezimazolam Besylate (UNII:280XQ6482H)

Forodesine Hydrochloride: Forodesine (CHEBI:43362)

Colistin Sulfate: Various, this one might need some work

Canagliflozin Hydrate: Canagliflozen (CHEBI:73272)

Albuterol Sulfate: Salbutamol (CHEBI:2549)

etc...

In [10]:
df[ (df["has_rxcui"] == True) & (df["in_robokop"] == False) & (df["combination"] == "FALSE") ]

,id,name,combination,nodenorm_id,has_rxcui,in_robokop
1,DRUGBANK:DB14113,ANTIVENIN (LATRODECTUS MACTANS),FALSE,DRUGBANK:DB14113,True,False
2,MESH:D016899,INTERFERON-BETA,FALSE,MESH:D016899,True,False
9,DRUGBANK:DB10558,PURIFIED PINEAPPLE STEM JUICE,FALSE,DRUGBANK:DB10558,True,False
10,DRUGBANK:DB13905,CENTRUROIDES (SCORPION) IMMUNE F(AB')2 (EQUINE...,FALSE,DRUGBANK:DB13905,True,False
17,DRUGBANK:DB10343,BCG LIVE,FALSE,DRUGBANK:DB10343,True,False
...,...,...,...,...,...,...
743,RXCUI:2639950,EFGARTIGIMOD ALFA AND HYALURONIDASE-QVFC,FALSE,RXCUI:2639950,True,False
753,UNII:C9R35M8XV6,"""PROTECTIVE ANTIGEN OF BORDETELLA PERTUSSIS DI...",FALSE,UNII:C9R35M8XV6,True,False
755,DRUGBANK:DB11294,COCCIDIOIDES IMMITIS SPHERULE-DERIVED SKIN TES...,FALSE,DRUGBANK:DB11294,True,False
764,DRUGBANK:DB16709,MATRIX-APPLIED CHARACTERISED AUTOLOGOUS CULTUR...,FALSE,DRUGBANK:DB16709,True,False


To be honest, a lot of these look like pretty crufty stuff. Antivenoms, bacteria, modified cells, gene therapies.
 But there are some that we might want to pay attention to.

For example, Interferon Beta.  In robokop we have more specific forms like interferon beta-1a.  Its a very large clique
including specific formulations like Avonex whereas the Inteferon clique just has (Mesh/UMLS/RXCUI). I think that there's
potentially problems here because people use the same string (Interferon beta) to mean different things and unless there
is a structure (which we dont have in mesh/umls/rxcui) you don't really know what they mean and whether you should
normalize them together.




